In [17]:
import nltk
import numpy
import json
import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import sklearn
import string
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import precision_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import randint
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
lemmatizer = WordNetLemmatizer()

#Current Classifier: 1 is Greeting, 2 is Thank you, 3 is Product information

df = pd.read_csv("intentdata.csv")
df

,Pattern,Greetings,Thanks,Product,Shipping Times,Payments,Refund
0,Hello!,1,0,0,0,0,0
1,Hi there!,1,0,0,0,0,0
2,Good morning!,1,0,0,0,0,0
3,Good afternoon!,1,0,0,0,0,0
4,Good evening!,1,0,0,0,0,0
...,...,...,...,...,...,...,...
342,Can I receive a refund for this product if it ...,0,0,0,0,0,1
343,Can you tell me the procedure for returning a ...,0,0,0,0,0,1
344,Can I receive a refund for this item if I no l...,0,0,0,0,0,1
345,How long does it take to receive a refund for ...,0,0,0,0,0,1


In [18]:
# Used from the d2l.
def clean(doc): # doc is a string of text
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([lemmatizer.lemmatize(word.lower()) for word in doc.split()])
    # remove punctuation and numbers
    return doc

In [19]:
#Preprocess the pandas dataframe
df['Pattern'] = df["Pattern"].apply(clean)

In [20]:
vect = CountVectorizer()
vect.fit(df["Pattern"])

X = vect.transform(df["Pattern"])
dfnew = pd.DataFrame(X.toarray(), columns=vect.get_feature_names_out())
dfnew['Greetings'] = df['Greetings']
dfnew['Thanks'] = df['Thanks']
dfnew['Product'] = df['Product']
dfnew['Shipping Times'] = df['Shipping Times']
dfnew['Payments'] = df['Payments']
dfnew['Refund'] = df['Refund']

In [21]:
print(sorted(vect.vocabulary_))
with open('vocabulary.txt', 'w') as filehandle:
    json.dump(sorted(vect.vocabulary_), filehandle)

['about', 'accept', 'accepted', 'access', 'accidentally', 'account', 'add', 'additional', 'address', 'after', 'afternoon', 'again', 'ahoy', 'already', 'am', 'amazing', 'amount', 'an', 'and', 'anticipate', 'anticipated', 'any', 'app', 'apple', 'appreciate', 'appreciative', 'approval', 'are', 'area', 'arrival', 'arrive', 'arrived', 'assist', 'associated', 'at', 'atm', 'automatic', 'available', 'awesome', 'back', 'backordered', 'bad', 'bank', 'be', 'been', 'begin', 'best', 'better', 'between', 'billing', 'bottom', 'bought', 'bundle', 'buy', 'buyer', 'by', 'can', 'cancel', 'cannot', 'cant', 'card', 'carrier', 'carry', 'cash', 'cashback', 'cashier', 'catch', 'category', 'center', 'certain', 'change', 'changed', 'chat', 'check', 'choose', 'clearance', 'company', 'complete', 'condition', 'confirmation', 'contact', 'corporate', 'cost', 'could', 'couldnt', 'country', 'credit', 'cryptocurrency', 'currency', 'current', 'custom', 'customer', 'damage', 'damaged', 'date', 'day', 'deadline', 'deal', 

In [22]:
dfnew

,about,accept,accepted,access,accidentally,account,add,additional,address,after,...,your,youre,youve,zelle,Greetings,Thanks,Product,Shipping Times,Payments,Refund
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
343,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
344,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
345,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [23]:
X = dfnew.drop('Greetings',axis=1)
X = X.drop('Thanks',axis=1)
X = X.drop('Product',axis=1)
X = X.drop('Shipping Times',axis=1)
X = X.drop('Payments',axis=1)
X = X.drop('Refund',axis=1)
y = dfnew[['Greetings','Thanks','Product', 'Shipping Times', 'Payments', 'Refund']]
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size = 0.2)

In [24]:
X

,about,accept,accepted,access,accidentally,account,add,additional,address,after,...,work,world,wrong,year,yo,you,your,youre,youve,zelle
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
343,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
344,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
345,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
y

,Greetings,Thanks,Product,Shipping Times,Payments,Refund
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
342,0,0,0,0,0,1
343,0,0,0,0,0,1
344,0,0,0,0,0,1
345,0,0,0,0,0,1


In [26]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [27]:
X_train.shape

(277, 467)

In [28]:
y_train.shape

(277, 6)

In [29]:
"""
print("--Make model--")
model = Sequential()
model.add(Dense(32,input_shape=(85,),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense())
"""
print("--Make model--")
model = tf.keras.models.Sequential([
  tf.keras.Input(shape=(X_train.shape[1],)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

--Make model--
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 32)                14976     
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 64)                2112      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 6)                 390       
                                                                 
Total params: 17,478
Trainable params: 17,478
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
print("--Fit model--")
model.fit(X_train, y_train, epochs=100)

--Fit model--
Epoch 1/100
9/9 [==============================] - 1s 3ms/step - loss: 1.7349 - accuracy: 0.2780
Epoch 2/100
9/9 [==============================] - 0s 3ms/step - loss: 1.7097 - accuracy: 0.2310
Epoch 3/100
9/9 [==============================] - 0s 7ms/step - loss: 1.6849 - accuracy: 0.2708
Epoch 4/100
9/9 [==============================] - 0s 7ms/step - loss: 1.6087 - accuracy: 0.3466
Epoch 5/100
9/9 [==============================] - 0s 4ms/step - loss: 1.5543 - accuracy: 0.3682
Epoch 6/100
9/9 [==============================] - 0s 4ms/step - loss: 1.4928 - accuracy: 0.4043
Epoch 7/100
9/9 [==============================] - 0s 3ms/step - loss: 1.4500 - accuracy: 0.4513
Epoch 8/100
9/9 [==============================] - 0s 3ms/step - loss: 1.3672 - accuracy: 0.5307
Epoch 9/100
9/9 [==============================] - 0s 2ms/step - loss: 1.3212 - accuracy: 0.5343
Epoch 10/100
9/9 [==============================] - 0s 3ms/step - loss: 1.2447 - accuracy: 0.5921
Epoch 11/100
9/

9/9 [==============================] - 0s 3ms/step - loss: 0.0955 - accuracy: 0.9819
Epoch 85/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0808 - accuracy: 0.9711
Epoch 86/100
9/9 [==============================] - 0s 2ms/step - loss: 0.0620 - accuracy: 0.9892
Epoch 87/100
9/9 [==============================] - 0s 2ms/step - loss: 0.0621 - accuracy: 0.9856
Epoch 88/100
9/9 [==============================] - 0s 2ms/step - loss: 0.0610 - accuracy: 0.9856
Epoch 89/100
9/9 [==============================] - 0s 2ms/step - loss: 0.0566 - accuracy: 0.9892
Epoch 90/100
9/9 [==============================] - 0s 2ms/step - loss: 0.0711 - accuracy: 0.9856
Epoch 91/100
9/9 [==============================] - 0s 2ms/step - loss: 0.0913 - accuracy: 0.9675
Epoch 92/100
9/9 [==============================] - 0s 2ms/step - loss: 0.0614 - accuracy: 0.9892
Epoch 93/100
9/9 [==============================] - 0s 2ms/step - loss: 0.0565 - accuracy: 0.9892
Epoch 94/100
9/9 [===============

In [31]:
print("--Evaluate model--")
model_loss1, model_acc1 = model.evaluate(X_train,  y_train, verbose=2)
model_loss2, model_acc2 = model.evaluate(X_test,  y_test, verbose=2)
print(f"Train / Test Accuracy: {model_acc1*100:.1f}% / {model_acc2*100:.1f}%")

--Evaluate model--
9/9 - 0s - loss: 0.0046 - accuracy: 1.0000 - 133ms/epoch - 15ms/step
3/3 - 0s - loss: 0.1976 - accuracy: 0.9429 - 27ms/epoch - 9ms/step
Train / Test Accuracy: 100.0% / 94.3%


In [32]:
model.save('chatbot_WIP.h5')